## **Gradio Day!**
Today we will build User Interfaces using the outrageously simple Gradio framework.

Prepare for joy!

Please note: your Gradio screens may appear in 'dark mode' or 'light mode' depending on your computer settings.

In [2]:
import os 
import requests 
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
from ollama import Client
from groq import Groq


In [3]:
# importing gradio

import gradio as gr

d:\LLM_Engineering\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
load_dotenv(override=True)

python-dotenv could not parse statement starting at line 11


True

In [5]:
openai = OpenAI()
client = Client()
groq = Groq()

In [6]:
# A generic system message

system_message = "You are a helpful assistant."

In [7]:
# Let's wrap a call to GPT 4.1 nano in a simple function

def message_gpt(prompt):
    messages = [
        {'role': 'system', 'content': system_message}, 
        {'role': 'user', 'content': prompt}
    ]
    completion = openai.chat.completions.create(
        model = 'gpt-4.1-nano',
        messages=messages, 
    )
    return completion.choices[0].message.content

In [8]:
message_gpt("what is the capital of france?")

'The capital of France is Paris.'

In [20]:
# a simple function

def shout(text: str):
    print(f"Shout has been called with input: ", text)
    return text.upper()

In [10]:
shout('hello')

'HELLO'

### **User Interface Time**

In [12]:
view = gr.Interface(fn=shout, inputs="textbox", outputs="textbox").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Created dataset file at: .gradio\flagged\dataset1.csv


In [16]:
view = gr.Interface(fn=shout, inputs="textbox", outputs="textbox", allow_flagging="never")
view.launch()

d:\LLM_Engineering\.venv\Lib\site-packages\gradio\interface.py:425: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


In [21]:
view = gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never")
view.launch(share=True)

* Running on local URL:  http://127.0.0.1:7868

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


Shout has been called with input:  hi there


In [22]:
view = gr.Interface(
    fn=shout,
    inputs=[gr.Textbox(label="Your Message: ", lines=6)], 
    outputs=[gr.Textbox(label="Response", lines=8)],
    flagging_mode="never"
)
view.launch(share=True)

* Running on local URL:  http://127.0.0.1:7869

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


Shout has been called with input:  hi there



In [23]:
# Now with our boy GPT

view = gr.Interface(
    fn=message_gpt,
    inputs=[gr.Textbox(label="Your Message: ", lines=6)], 
    outputs=[gr.Textbox(label="Response", lines=8)],
    flagging_mode="never"
)
view.launch(share=True)

* Running on local URL:  http://127.0.0.1:7870

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


In [26]:
system_message="You are a helpful assistant that responds in Markdown."

view = gr.Interface(
    fn=message_gpt, 
    inputs=[gr.Textbox(label="Whats on your mind!!", lines=2)], 
    outputs=[gr.Markdown(label="Assistant's Response")], 
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7872
* To create a public link, set `share=True` in `launch()`.


In [34]:
# let's create a call that streams back results

def stream_llama(prompt):
    messages = [
        {'role': 'system', 'content': system_message}, 
        {'role': 'user', 'content': prompt}
    ]

    stream = groq.chat.completions.create(
        model='llama-3.3-70b-versatile',
        messages=messages,
        stream=True                    
    )

    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [35]:
view = gr.Interface(
    fn=stream_llama, 
    inputs=[gr.Textbox(label="Your message", lines=2)], 
    outputs=[gr.Markdown(label="Response")], 
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7876
* To create a public link, set `share=True` in `launch()`.


In [ ]:
# Stream the results from GPT

def stream_gpt(prompt):
    messages = [
        {'role': 'system', 'content': system_message}, 
        {'role': 'user', 'content': prompt}
    ]

    stream = openai.chat.completions.create(
        model='gpt-4.1-nano',
        messages=messages,
        stream=True                    
    )

    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

### Multiple Variables in UI

In [40]:
# Now a function with some variables

def stream_model(prompt, model):
    if model == 'GPT':
        result = stream_gpt(prompt)
    elif model == 'llama':
        result = stream_llama(prompt)
    else:
        raise ValueError("unknown model")
    for chunk in result:
        yield chunk

In [43]:
view = gr.Interface(
    fn=stream_model, 
    inputs=[gr.Textbox(label="Your message"), gr.Dropdown(["GPT", "llama"], label="Select model")], 
    outputs=[gr.Markdown(label="Response")], 
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7881
* To create a public link, set `share=True` in `launch()`.


### Building a Company Brochure Generator

In [44]:
# A class to represent a Webpage

class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [45]:
system_message = "You are an assistant that analyzes the contents of a company website landing page \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown."

In [48]:
def stream_brochure(company_name, url, model):
    prompt = f"Please generate a company brochure for {company_name}. \
        Here is their landing page:\n"
    prompt += Website(url).get_contents()

    if model == "GPT":
        result = stream_gpt(prompt)
    elif model == "llama":
        result = stream_llama(prompt)
    else:
        raise ValueError("Unknown model")
    
    yield from result
    

Previously, it had these lines:
```Python
for chunk in result:
  yield chunk
```
There's actually a more elegant way to achieve this (which Python people might call more 'Pythonic'):

```python 
yield from result
```

In [49]:
view = gr.Interface(
    fn=stream_brochure,
    inputs=[
        gr.Textbox(label="Company name:"),
        gr.Textbox(label="Landing page URL including http:// or https://"),
        gr.Dropdown(["GPT", "llama"], label="Select model")],
    outputs=[gr.Markdown(label="Brochure:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7883
* To create a public link, set `share=True` in `launch()`.
